In [1]:
# pip install sodapy

In [2]:
import json
import requests
import pandas as pd
from pprint import pprint
from sodapy import Socrata
from config import SODAPY_APPTOKEN
from config_db import password_db
import os
import datetime as dt
from sqlalchemy import create_engine
import calendar

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [3]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.bts.gov",
                 SODAPY_APPTOKEN) #add your API token here

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.bts.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("w96p-f2qv", Level = "State", limit=100000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,level,date,state_fips,state_code,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,trips_1_3,trips_3_5,trips_5_10,trips_10_25,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500
0,State,2020-09-11T00:00:00.000,16,ID,415095.0,1339113.0,4979774.0,1132127.0,1299923.0,598257.0,720970.0,713064.0,275469.0,132484.0,80661.0,19167.0,7652.0
1,State,2020-09-11T00:00:00.000,41,OR,1221063.0,2969650.0,11347154.0,2713552.0,3009503.0,1398978.0,1623881.0,1649200.0,551544.0,240625.0,107893.0,26924.0,25054.0
2,State,2020-09-11T00:00:00.000,17,IL,3283326.0,9457754.0,43667012.0,10540371.0,10547870.0,5102596.0,6573051.0,6908445.0,2636019.0,772368.0,412957.0,105652.0,67683.0
3,State,2020-09-11T00:00:00.000,40,OK,895283.0,3047796.0,12360147.0,2793229.0,3073565.0,1456721.0,1837300.0,1999598.0,703691.0,306094.0,154593.0,26988.0,8368.0
4,State,2020-09-11T00:00:00.000,18,IN,1512392.0,5179486.0,23700960.0,5322707.0,5863168.0,2906604.0,3669836.0,3730834.0,1364000.0,504648.0,253265.0,54614.0,31284.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31666,State,2020-07-05T00:00:00.000,51,VA,2041257,6476428,24337162,6468923,5883294,3024789,3626618,3207172,1103602,570564,346351,68862,36987
31667,State,2020-07-05T00:00:00.000,53,WA,2039939,5495652,18644796,5485722,4384396,2075740,2574619,2540516,898198,431053,182312,31601,40639
31668,State,2020-07-05T00:00:00.000,54,WV,438868,1366964,4957656,1152574,1257178,625621,784140,698114,248782,119493,58682,9466,3606
31669,State,2020-07-05T00:00:00.000,55,WI,1359309,4454259,15539388,3917674,3708884,1875900,2274045,2119498,831156,468478,281667,43359,18727


In [4]:
# sort dates to analyze
results_date_sort = results_df.sort_values('date')
results_date_sort

,level,date,state_fips,state_code,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,trips_1_3,trips_3_5,trips_5_10,trips_10_25,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500
3519,State,2019-01-01T00:00:00.000,01,AL,1028578,3844356,11968328,2792091,3151108,1604875,1879113,1629105,534229,232409,108187,27164,10047
3548,State,2019-01-01T00:00:00.000,33,NH,312645,1039666,3705345,726417,970009,492350,582185,573171,231818,90462,27333,3998,7602
3549,State,2019-01-01T00:00:00.000,34,NJ,2244189,6637104,26078674,7491874,6764617,3009746,3582118,3262723,1233743,486493,155179,28397,63784
3550,State,2019-01-01T00:00:00.000,35,NM,525761,1563262,6475422,1972284,1756894,795723,834525,632375,231297,139256,83929,19001,10138
3551,State,2019-01-01T00:00:00.000,36,NY,5931873,13550603,60957950,18317662,15401822,7187598,8789865,7589390,2294488,857637,329795,72132,117561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,State,2020-09-12T00:00:00.000,13,GA,2531945.0,7987530.0,30701584.0,6440848.0,7397919.0,3851558.0,5116353.0,4868580.0,1826130.0,673854.0,373991.0,105617.0,46734.0
240,State,2020-09-12T00:00:00.000,36,NY,6579119.0,12963090.0,64920430.0,17877099.0,16189164.0,7773281.0,9805686.0,8964172.0,2630089.0,1062262.0,438178.0,78363.0,102136.0
239,State,2020-09-12T00:00:00.000,44,RI,290914.0,766401.0,3312803.0,820947.0,905972.0,412754.0,508141.0,453198.0,150369.0,42029.0,11744.0,3915.0,3734.0
257,State,2020-09-12T00:00:00.000,08,CO,1663763.0,4031801.0,18217634.0,4467696.0,4488973.0,2195504.0,2746467.0,2733149.0,878044.0,423411.0,181583.0,49767.0,53040.0


In [5]:
# derive date into year/month/day columns
results_date_sort['year'] = pd.DatetimeIndex(results_date_sort['date']).year
results_date_sort['month'] = pd.DatetimeIndex(results_date_sort['date']).month
results_date_sort['day'] = pd.DatetimeIndex(results_date_sort['date']).day

results_date_sort

,level,date,state_fips,state_code,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,trips_1_3,trips_3_5,trips_5_10,trips_10_25,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500,year,month,day
3519,State,2019-01-01T00:00:00.000,01,AL,1028578,3844356,11968328,2792091,3151108,1604875,1879113,1629105,534229,232409,108187,27164,10047,2019,1,1
3548,State,2019-01-01T00:00:00.000,33,NH,312645,1039666,3705345,726417,970009,492350,582185,573171,231818,90462,27333,3998,7602,2019,1,1
3549,State,2019-01-01T00:00:00.000,34,NJ,2244189,6637104,26078674,7491874,6764617,3009746,3582118,3262723,1233743,486493,155179,28397,63784,2019,1,1
3550,State,2019-01-01T00:00:00.000,35,NM,525761,1563262,6475422,1972284,1756894,795723,834525,632375,231297,139256,83929,19001,10138,2019,1,1
3551,State,2019-01-01T00:00:00.000,36,NY,5931873,13550603,60957950,18317662,15401822,7187598,8789865,7589390,2294488,857637,329795,72132,117561,2019,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,State,2020-09-12T00:00:00.000,13,GA,2531945.0,7987530.0,30701584.0,6440848.0,7397919.0,3851558.0,5116353.0,4868580.0,1826130.0,673854.0,373991.0,105617.0,46734.0,2020,9,12
240,State,2020-09-12T00:00:00.000,36,NY,6579119.0,12963090.0,64920430.0,17877099.0,16189164.0,7773281.0,9805686.0,8964172.0,2630089.0,1062262.0,438178.0,78363.0,102136.0,2020,9,12
239,State,2020-09-12T00:00:00.000,44,RI,290914.0,766401.0,3312803.0,820947.0,905972.0,412754.0,508141.0,453198.0,150369.0,42029.0,11744.0,3915.0,3734.0,2020,9,12
257,State,2020-09-12T00:00:00.000,08,CO,1663763.0,4031801.0,18217634.0,4467696.0,4488973.0,2195504.0,2746467.0,2733149.0,878044.0,423411.0,181583.0,49767.0,53040.0,2020,9,12


In [6]:
# results_date_sort['month']= calendar.month_abbr['month_number']
results_date_sort['month_abbr'] = results_date_sort['month'].apply(lambda x: calendar.month_abbr[x])
# results_date_sort['results_date_sort'] = calendar.month_abbr['month'] 

# df.date = df.date.apply(lambda x: datetime.strptime(x, "%d-%M-%Y"))
# df.date = df.date.apply(lambda x: datetime.strftime(x, "%d-%b-%Y").upper())

# results_date_sort['date']= results_date_sort['date'].str.split('T', n = 1, expand = True)

results_date_sort


,level,date,state_fips,state_code,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,trips_1_3,trips_3_5,...,trips_10_25,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500,year,month,day,month_abbr
3519,State,2019-01-01T00:00:00.000,01,AL,1028578,3844356,11968328,2792091,3151108,1604875,...,1629105,534229,232409,108187,27164,10047,2019,1,1,Jan
3548,State,2019-01-01T00:00:00.000,33,NH,312645,1039666,3705345,726417,970009,492350,...,573171,231818,90462,27333,3998,7602,2019,1,1,Jan
3549,State,2019-01-01T00:00:00.000,34,NJ,2244189,6637104,26078674,7491874,6764617,3009746,...,3262723,1233743,486493,155179,28397,63784,2019,1,1,Jan
3550,State,2019-01-01T00:00:00.000,35,NM,525761,1563262,6475422,1972284,1756894,795723,...,632375,231297,139256,83929,19001,10138,2019,1,1,Jan
3551,State,2019-01-01T00:00:00.000,36,NY,5931873,13550603,60957950,18317662,15401822,7187598,...,7589390,2294488,857637,329795,72132,117561,2019,1,1,Jan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,State,2020-09-12T00:00:00.000,13,GA,2531945.0,7987530.0,30701584.0,6440848.0,7397919.0,3851558.0,...,4868580.0,1826130.0,673854.0,373991.0,105617.0,46734.0,2020,9,12,Sep
240,State,2020-09-12T00:00:00.000,36,NY,6579119.0,12963090.0,64920430.0,17877099.0,16189164.0,7773281.0,...,8964172.0,2630089.0,1062262.0,438178.0,78363.0,102136.0,2020,9,12,Sep
239,State,2020-09-12T00:00:00.000,44,RI,290914.0,766401.0,3312803.0,820947.0,905972.0,412754.0,...,453198.0,150369.0,42029.0,11744.0,3915.0,3734.0,2020,9,12,Sep
257,State,2020-09-12T00:00:00.000,08,CO,1663763.0,4031801.0,18217634.0,4467696.0,4488973.0,2195504.0,...,2733149.0,878044.0,423411.0,181583.0,49767.0,53040.0,2020,9,12,Sep


In [7]:

results_date_sort['date'] = pd.to_datetime(results_date_sort['date'], dayfirst=True)
results_date_sort['date'] = results_date_sort['date'].dt.strftime('%d-%b-%Y')

results_date_sort

,level,date,state_fips,state_code,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,trips_1_3,trips_3_5,...,trips_10_25,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500,year,month,day,month_abbr
3519,State,01-Jan-2019,01,AL,1028578,3844356,11968328,2792091,3151108,1604875,...,1629105,534229,232409,108187,27164,10047,2019,1,1,Jan
3548,State,01-Jan-2019,33,NH,312645,1039666,3705345,726417,970009,492350,...,573171,231818,90462,27333,3998,7602,2019,1,1,Jan
3549,State,01-Jan-2019,34,NJ,2244189,6637104,26078674,7491874,6764617,3009746,...,3262723,1233743,486493,155179,28397,63784,2019,1,1,Jan
3550,State,01-Jan-2019,35,NM,525761,1563262,6475422,1972284,1756894,795723,...,632375,231297,139256,83929,19001,10138,2019,1,1,Jan
3551,State,01-Jan-2019,36,NY,5931873,13550603,60957950,18317662,15401822,7187598,...,7589390,2294488,857637,329795,72132,117561,2019,1,1,Jan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,State,12-Sep-2020,13,GA,2531945.0,7987530.0,30701584.0,6440848.0,7397919.0,3851558.0,...,4868580.0,1826130.0,673854.0,373991.0,105617.0,46734.0,2020,9,12,Sep
240,State,12-Sep-2020,36,NY,6579119.0,12963090.0,64920430.0,17877099.0,16189164.0,7773281.0,...,8964172.0,2630089.0,1062262.0,438178.0,78363.0,102136.0,2020,9,12,Sep
239,State,12-Sep-2020,44,RI,290914.0,766401.0,3312803.0,820947.0,905972.0,412754.0,...,453198.0,150369.0,42029.0,11744.0,3915.0,3734.0,2020,9,12,Sep
257,State,12-Sep-2020,08,CO,1663763.0,4031801.0,18217634.0,4467696.0,4488973.0,2195504.0,...,2733149.0,878044.0,423411.0,181583.0,49767.0,53040.0,2020,9,12,Sep


In [8]:
# verify datatypes
results_date_sort.dtypes

level                   object
date                    object
state_fips              object
state_code              object
pop_stay_at_home        object
pop_not_stay_at_home    object
trips                   object
trips_1                 object
trips_1_3               object
trips_3_5               object
trips_5_10              object
trips_10_25             object
trips_25_50             object
trips_50_100            object
trips_100_250           object
trips_250_500           object
trips_500               object
year                     int64
month                    int64
day                      int64
month_abbr              object
dtype: object

In [9]:
# copy dataset into new dataset
results_convert = results_date_sort


In [10]:
# convert datatypes
results_convert['level'] = results_date_sort['level'].astype('str')
results_convert['date'] = results_date_sort['date'].astype('str')
results_convert['state_fips'] = results_date_sort['state_fips'].astype('int')
results_convert['state_code'] = results_date_sort['state_code'].astype('str')
results_convert['pop_stay_at_home'] = results_date_sort['pop_stay_at_home'].astype('float').astype('int')
results_convert['pop_not_stay_at_home'] = results_date_sort['pop_not_stay_at_home'].astype('float').astype('int')
results_convert['trips'] = results_date_sort['trips'].astype('float').astype('int')
results_convert['trips_1'] = results_date_sort['trips_1'].astype('float').astype('int')
results_convert['trips_1_3'] = results_date_sort['trips_1_3'].astype('float').astype('int')
results_convert['trips_3_5'] = results_date_sort['trips_3_5'].astype('float').astype('int')
results_convert['trips_5_10'] = results_date_sort['trips_5_10'].astype('float').astype('int')
results_convert['trips_10_25'] = results_date_sort['trips_10_25'].astype('float').astype('int')
results_convert['trips_25_50'] = results_date_sort['trips_25_50'].astype('float').astype('int')
results_convert['trips_50_100'] = results_date_sort['trips_50_100'].astype('float').astype('int')
results_convert['trips_100_250'] = results_date_sort['trips_100_250'].astype('float').astype('int')
results_convert['trips_250_500'] = results_date_sort['trips_250_500'].astype('float').astype('int')
results_convert['trips_500'] = results_date_sort['trips_500'].astype('float').astype('int')
results_convert['year'] = results_date_sort['year'].astype('str')
results_convert['month'] = results_date_sort['month'].astype('str')
results_convert['day'] = results_date_sort['day'].astype('str')

results_convert

,level,date,state_fips,state_code,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,trips_1_3,trips_3_5,...,trips_10_25,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500,year,month,day,month_abbr
3519,State,01-Jan-2019,1,AL,1028578,3844356,11968328,2792091,3151108,1604875,...,1629105,534229,232409,108187,27164,10047,2019,1,1,Jan
3548,State,01-Jan-2019,33,NH,312645,1039666,3705345,726417,970009,492350,...,573171,231818,90462,27333,3998,7602,2019,1,1,Jan
3549,State,01-Jan-2019,34,NJ,2244189,6637104,26078674,7491874,6764617,3009746,...,3262723,1233743,486493,155179,28397,63784,2019,1,1,Jan
3550,State,01-Jan-2019,35,NM,525761,1563262,6475422,1972284,1756894,795723,...,632375,231297,139256,83929,19001,10138,2019,1,1,Jan
3551,State,01-Jan-2019,36,NY,5931873,13550603,60957950,18317662,15401822,7187598,...,7589390,2294488,857637,329795,72132,117561,2019,1,1,Jan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,State,12-Sep-2020,13,GA,2531945,7987530,30701584,6440848,7397919,3851558,...,4868580,1826130,673854,373991,105617,46734,2020,9,12,Sep
240,State,12-Sep-2020,36,NY,6579119,12963090,64920430,17877099,16189164,7773281,...,8964172,2630089,1062262,438178,78363,102136,2020,9,12,Sep
239,State,12-Sep-2020,44,RI,290914,766401,3312803,820947,905972,412754,...,453198,150369,42029,11744,3915,3734,2020,9,12,Sep
257,State,12-Sep-2020,8,CO,1663763,4031801,18217634,4467696,4488973,2195504,...,2733149,878044,423411,181583,49767,53040,2020,9,12,Sep


In [11]:
# concatenate month_abbr and year
results_convert['month_year'] = results_convert['month_abbr'] + '-' + results_convert['year']
results_convert

,level,date,state_fips,state_code,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,trips_1_3,trips_3_5,...,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500,year,month,day,month_abbr,month_year
3519,State,01-Jan-2019,1,AL,1028578,3844356,11968328,2792091,3151108,1604875,...,534229,232409,108187,27164,10047,2019,1,1,Jan,Jan-2019
3548,State,01-Jan-2019,33,NH,312645,1039666,3705345,726417,970009,492350,...,231818,90462,27333,3998,7602,2019,1,1,Jan,Jan-2019
3549,State,01-Jan-2019,34,NJ,2244189,6637104,26078674,7491874,6764617,3009746,...,1233743,486493,155179,28397,63784,2019,1,1,Jan,Jan-2019
3550,State,01-Jan-2019,35,NM,525761,1563262,6475422,1972284,1756894,795723,...,231297,139256,83929,19001,10138,2019,1,1,Jan,Jan-2019
3551,State,01-Jan-2019,36,NY,5931873,13550603,60957950,18317662,15401822,7187598,...,2294488,857637,329795,72132,117561,2019,1,1,Jan,Jan-2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,State,12-Sep-2020,13,GA,2531945,7987530,30701584,6440848,7397919,3851558,...,1826130,673854,373991,105617,46734,2020,9,12,Sep,Sep-2020
240,State,12-Sep-2020,36,NY,6579119,12963090,64920430,17877099,16189164,7773281,...,2630089,1062262,438178,78363,102136,2020,9,12,Sep,Sep-2020
239,State,12-Sep-2020,44,RI,290914,766401,3312803,820947,905972,412754,...,150369,42029,11744,3915,3734,2020,9,12,Sep,Sep-2020
257,State,12-Sep-2020,8,CO,1663763,4031801,18217634,4467696,4488973,2195504,...,878044,423411,181583,49767,53040,2020,9,12,Sep,Sep-2020


In [12]:
# webscrape coordinates for geomapping
url_coord = 'https://inkplant.com/code/state-latitudes-longitudes'
table_coord = pd.read_html(url_coord)
table_coord


[                   State   Latitude   Longitude
 0                Alabama  32.806671  -86.791130
 1                 Alaska  61.370716 -152.404419
 2                Arizona  33.729759 -111.431221
 3               Arkansas  34.969704  -92.373123
 4             California  36.116203 -119.681564
 5               Colorado  39.059811 -105.311104
 6            Connecticut  41.597782  -72.755371
 7               Delaware  39.318523  -75.507141
 8   District of Columbia  38.897438  -77.026817
 9                Florida  27.766279  -81.686783
 10               Georgia  33.040619  -83.643074
 11                Hawaii  21.094318 -157.498337
 12                 Idaho  44.240459 -114.478828
 13              Illinois  40.349457  -88.986137
 14               Indiana  39.849426  -86.258278
 15                  Iowa  42.011539  -93.210526
 16                Kansas  38.526600  -96.726486
 17              Kentucky  37.668140  -84.670067
 18             Louisiana  31.169546  -91.867805
 19                 

In [13]:
# get datascrapped into dataframe
df_coord = table_coord[0]
df_coord['Latitude'] = round(df_coord['Latitude'],6)
df_coord['Longitude'] = round(df_coord['Longitude'],6)

df_coord.head()

,State,Latitude,Longitude
0,Alabama,32.806671,-86.791130
1,Alaska,61.370716,-152.404419
2,Arizona,33.729759,-111.431221
3,Arkansas,34.969704,-92.373123
4,California,36.116203,-119.681564


In [14]:
# webscrape State short code in order to join with transactional data
url_states ='https://www.factmonster.com/us/postal-information/state-abbreviations-and-state-postal-codes'
table_state = pd.read_html(url_states)
table_state

[                State Abbreviation Postalcode
 0             Alabama         Ala.         AL
 1              Alaska       Alaska         AK
 2      American Samoa          NaN         AS
 3             Arizona        Ariz.         AZ
 4            Arkansas         Ark.         AR
 5          California       Calif.         CA
 6            Colorado        Colo.         CO
 7         Connecticut        Conn.         CT
 8            Delaware         Del.         DE
 9   Dist. of Columbia         D.C.         DC
 10            Florida         Fla.         FL
 11            Georgia          Ga.         GA
 12               Guam         Guam         GU
 13             Hawaii       Hawaii         HI
 14              Idaho        Idaho         ID
 15           Illinois         Ill.         IL
 16            Indiana         Ind.         IN
 17               Iowa         Iowa         IA
 18             Kansas        Kans.         KS
 19           Kentucky          Ky.         KY
 20          

In [15]:
# convert state dataset into dataframe
df_state = table_state[0]
df_state.head()

,State,Abbreviation,Postalcode
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,American Samoa,NaN,AS
3,Arizona,Ariz.,AZ
4,Arkansas,Ark.,AR


In [16]:
# drop Abbreviation column
df_state = df_state.drop(['Abbreviation'],axis=1)
df_state.head()

,State,Postalcode
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [17]:
# merge state dataset and coordinates dataset
merged_state = pd.merge(df_coord,df_state, how = 'inner', on='State')
merged_state

,State,Latitude,Longitude,Postalcode
0,Alabama,32.806671,-86.791130,AL
1,Alaska,61.370716,-152.404419,AK
2,Arizona,33.729759,-111.431221,AZ
3,Arkansas,34.969704,-92.373123,AR
4,California,36.116203,-119.681564,CA
5,Colorado,39.059811,-105.311104,CO
6,Connecticut,41.597782,-72.755371,CT
7,Delaware,39.318523,-75.507141,DE
8,Florida,27.766279,-81.686783,FL
9,Georgia,33.040619,-83.643074,GA


In [18]:
# rename columns prior to join to transactional data
merged_state = merged_state.rename(columns={'Postalcode':'state_code','State':'state','Latitude':'latitude',
                                            'Longitude':'longitude'})
merged_state

,state,latitude,longitude,state_code
0,Alabama,32.806671,-86.791130,AL
1,Alaska,61.370716,-152.404419,AK
2,Arizona,33.729759,-111.431221,AZ
3,Arkansas,34.969704,-92.373123,AR
4,California,36.116203,-119.681564,CA
5,Colorado,39.059811,-105.311104,CO
6,Connecticut,41.597782,-72.755371,CT
7,Delaware,39.318523,-75.507141,DE
8,Florida,27.766279,-81.686783,FL
9,Georgia,33.040619,-83.643074,GA


In [19]:
# merge state/coordinates dataset with transactional data
merged_travel = pd.merge(results_convert,merged_state, how = 'left', on='state_code')
merged_travel

,level,date,state_fips,state_code,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,trips_1_3,trips_3_5,...,trips_250_500,trips_500,year,month,day,month_abbr,month_year,state,latitude,longitude
0,State,01-Jan-2019,1,AL,1028578,3844356,11968328,2792091,3151108,1604875,...,27164,10047,2019,1,1,Jan,Jan-2019,Alabama,32.806671,-86.791130
1,State,01-Jan-2019,33,NH,312645,1039666,3705345,726417,970009,492350,...,3998,7602,2019,1,1,Jan,Jan-2019,New Hampshire,43.452492,-71.563896
2,State,01-Jan-2019,34,NJ,2244189,6637104,26078674,7491874,6764617,3009746,...,28397,63784,2019,1,1,Jan,Jan-2019,New Jersey,40.298904,-74.521011
3,State,01-Jan-2019,35,NM,525761,1563262,6475422,1972284,1756894,795723,...,19001,10138,2019,1,1,Jan,Jan-2019,New Mexico,34.840515,-106.248482
4,State,01-Jan-2019,36,NY,5931873,13550603,60957950,18317662,15401822,7187598,...,72132,117561,2019,1,1,Jan,Jan-2019,New York,42.165726,-74.948051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31666,State,12-Sep-2020,13,GA,2531945,7987530,30701584,6440848,7397919,3851558,...,105617,46734,2020,9,12,Sep,Sep-2020,Georgia,33.040619,-83.643074
31667,State,12-Sep-2020,36,NY,6579119,12963090,64920430,17877099,16189164,7773281,...,78363,102136,2020,9,12,Sep,Sep-2020,New York,42.165726,-74.948051
31668,State,12-Sep-2020,44,RI,290914,766401,3312803,820947,905972,412754,...,3915,3734,2020,9,12,Sep,Sep-2020,Rhode Island,41.680893,-71.511780
31669,State,12-Sep-2020,8,CO,1663763,4031801,18217634,4467696,4488973,2195504,...,49767,53040,2020,9,12,Sep,Sep-2020,Colorado,39.059811,-105.311104


In [20]:
# drop nan values
merged_travel.dropna(axis=0, how='any', inplace=True)
merged_travel

,level,date,state_fips,state_code,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,trips_1_3,trips_3_5,...,trips_250_500,trips_500,year,month,day,month_abbr,month_year,state,latitude,longitude
0,State,01-Jan-2019,1,AL,1028578,3844356,11968328,2792091,3151108,1604875,...,27164,10047,2019,1,1,Jan,Jan-2019,Alabama,32.806671,-86.791130
1,State,01-Jan-2019,33,NH,312645,1039666,3705345,726417,970009,492350,...,3998,7602,2019,1,1,Jan,Jan-2019,New Hampshire,43.452492,-71.563896
2,State,01-Jan-2019,34,NJ,2244189,6637104,26078674,7491874,6764617,3009746,...,28397,63784,2019,1,1,Jan,Jan-2019,New Jersey,40.298904,-74.521011
3,State,01-Jan-2019,35,NM,525761,1563262,6475422,1972284,1756894,795723,...,19001,10138,2019,1,1,Jan,Jan-2019,New Mexico,34.840515,-106.248482
4,State,01-Jan-2019,36,NY,5931873,13550603,60957950,18317662,15401822,7187598,...,72132,117561,2019,1,1,Jan,Jan-2019,New York,42.165726,-74.948051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31666,State,12-Sep-2020,13,GA,2531945,7987530,30701584,6440848,7397919,3851558,...,105617,46734,2020,9,12,Sep,Sep-2020,Georgia,33.040619,-83.643074
31667,State,12-Sep-2020,36,NY,6579119,12963090,64920430,17877099,16189164,7773281,...,78363,102136,2020,9,12,Sep,Sep-2020,New York,42.165726,-74.948051
31668,State,12-Sep-2020,44,RI,290914,766401,3312803,820947,905972,412754,...,3915,3734,2020,9,12,Sep,Sep-2020,Rhode Island,41.680893,-71.511780
31669,State,12-Sep-2020,8,CO,1663763,4031801,18217634,4467696,4488973,2195504,...,49767,53040,2020,9,12,Sep,Sep-2020,Colorado,39.059811,-105.311104


In [21]:
# drop time out of data values
# merged_travel['date']= merged_travel['date'].str.split('T', n = 1, expand = True)
# merged_travel

# concatenate month_abbr and year
# merged_travel['month_year'] = merged_travel['month_abbr'] + '-' + merged_travel['year']
# merged_travel

In [22]:
# drop level column
merged_travel = merged_travel.drop(['level'],axis=1)
merged_travel

,date,state_fips,state_code,pop_stay_at_home,pop_not_stay_at_home,trips,trips_1,trips_1_3,trips_3_5,trips_5_10,...,trips_250_500,trips_500,year,month,day,month_abbr,month_year,state,latitude,longitude
0,01-Jan-2019,1,AL,1028578,3844356,11968328,2792091,3151108,1604875,1879113,...,27164,10047,2019,1,1,Jan,Jan-2019,Alabama,32.806671,-86.791130
1,01-Jan-2019,33,NH,312645,1039666,3705345,726417,970009,492350,582185,...,3998,7602,2019,1,1,Jan,Jan-2019,New Hampshire,43.452492,-71.563896
2,01-Jan-2019,34,NJ,2244189,6637104,26078674,7491874,6764617,3009746,3582118,...,28397,63784,2019,1,1,Jan,Jan-2019,New Jersey,40.298904,-74.521011
3,01-Jan-2019,35,NM,525761,1563262,6475422,1972284,1756894,795723,834525,...,19001,10138,2019,1,1,Jan,Jan-2019,New Mexico,34.840515,-106.248482
4,01-Jan-2019,36,NY,5931873,13550603,60957950,18317662,15401822,7187598,8789865,...,72132,117561,2019,1,1,Jan,Jan-2019,New York,42.165726,-74.948051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31666,12-Sep-2020,13,GA,2531945,7987530,30701584,6440848,7397919,3851558,5116353,...,105617,46734,2020,9,12,Sep,Sep-2020,Georgia,33.040619,-83.643074
31667,12-Sep-2020,36,NY,6579119,12963090,64920430,17877099,16189164,7773281,9805686,...,78363,102136,2020,9,12,Sep,Sep-2020,New York,42.165726,-74.948051
31668,12-Sep-2020,44,RI,290914,766401,3312803,820947,905972,412754,508141,...,3915,3734,2020,9,12,Sep,Sep-2020,Rhode Island,41.680893,-71.511780
31669,12-Sep-2020,8,CO,1663763,4031801,18217634,4467696,4488973,2195504,2746467,...,49767,53040,2020,9,12,Sep,Sep-2020,Colorado,39.059811,-105.311104


In [23]:
# create dataset for initial validation and testing
merged_travel.to_csv('data/travel_data.csv')

In [24]:
# verify datatypes prior to drop data into database
merged_travel.dtypes

date                     object
state_fips                int32
state_code               object
pop_stay_at_home          int32
pop_not_stay_at_home      int32
trips                     int32
trips_1                   int32
trips_1_3                 int32
trips_3_5                 int32
trips_5_10                int32
trips_10_25               int32
trips_25_50               int32
trips_50_100              int32
trips_100_250             int32
trips_250_500             int32
trips_500                 int32
year                     object
month                    object
day                      object
month_abbr               object
month_year               object
state                    object
latitude                float64
longitude               float64
dtype: object

In [25]:
# create connection string to postgres
connection_string = 'postgres:'+password_db+'@localhost:5432/travel_data'

engine=create_engine(f"postgresql://{connection_string}")

In [26]:
# verify table created in database
engine.table_names()

['travel_data_details']

In [27]:
# push data into table already created in pgadmin
merged_travel.to_sql(name='travel_data_details', con=engine, if_exists='append', index=True)